## Lab | Revisiting Machine Learning Case Study

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)

In [38]:
data = pd.read_csv('learningSet.csv', low_memory=False)

### Complete the following steps on the categorical columns in the dataset:

Let's get the Categorical variables first:

In [39]:
categorical = data.select_dtypes(include=['object'])

### Check for null values in all the columns:
Let's create an additional DataFrame with the percentage of NaNs per column

In [40]:
nulls_percent_df = pd.DataFrame(categorical.isna().sum()/len(categorical)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

,column_name,nulls_percentage
0,OSOURCE,0.000000
1,STATE,0.000000
2,ZIP,0.000000
3,MAILCODE,0.000000
4,PVASTATE,0.000000
...,...,...
69,RFA_2A,0.000000
70,MDMAUD_R,0.000000
71,MDMAUD_F,0.000000
72,MDMAUD_A,0.000000


### Filtering those columns with NaN values

In [41]:
nulls_percent_df[nulls_percent_df['nulls_percentage']>0]

,column_name,nulls_percentage
73,GEOCODE2,0.001383


The only column with NaN values is GEOCODE2.

### Exclude the following variables by looking at the definitions. Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:

- OSOURCE - symbol definitions not provided, too many categories
- ZIP CODE - we are including state already

In [42]:
#Checking the actual name of the columns, it seems that "ZIP CODE" is just "ZIP"
nulls_percent_df[nulls_percent_df['column_name'].str.contains('OSOUR')]

#Excluding the aforementioned variables from the DataFrame:

categorical = categorical.drop(['OSOURCE', 'ZIP'], axis =1)

#Creating the list
drop_list = []

### Identify columns that are over 85% missing values:

In [25]:
nulls_percent_df[nulls_percent_df['nulls_percentage'] > 0.85]

,column_name,nulls_percentage


There is no column with more than 85% missing values.

### Remove those columns from the dataframe:

Let's better define a function to do all of this more efficiently:

In [26]:
def high_na_columns(df, threshold = 0.25): #by default we set 25% threshold, although we will input 85%
    
    nulls_percent_df = pd.DataFrame(df.isna().sum()/len(df)).reset_index()
    nulls_percent_df.columns = ['column_name', 'nulls_percentage']
    nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]
    columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>threshold]
    drop_columns_list = list(columns_above_threshold['column_name'])
    
    return drop_columns_list

Now let's apply the function and drop those columns:

In [27]:
drop_columns_list = high_na_columns(categorical, 0.85)

categorical = categorical.drop(columns = drop_columns_list, axis = 1)

In [33]:
drop_columns_list

[]

### Reduce the number of categories in the column GENDER. The column should only have either "M" for males, "F" for females, and "other" for all the rest:

Note that there are a few null values in the column. We will first replace those null values using the code below:

In [28]:
categorical['GENDER'] = categorical['GENDER'].fillna('F')

Let's start checking the value counts of this column:

In [29]:
categorical['GENDER'].value_counts(dropna=False)

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

Let's get a list of all the indices of those rows that are not Female, Male or NaN:

In [30]:
#This way we filter all those values which are not Female or Male:
#data[(data['GENDER'].isin(['F','M']) == False)]
#This way we filter as well the not NaN values:
#data[(data['GENDER'].isin(['F','M']) == False) & (data['GENDER'].notna())]['GENDER']

#So let's get a list of all the indices of those rows that are not Female, Male or NaN:
gen_indices =list(categorical[(categorical['GENDER'].isin(['F','M']) == False) & (categorical['GENDER'].notna())]['GENDER'].index)

Let's change the value of those rows from original gender classification to 'other':

In [31]:
categorical.iloc[gen_indices,list(categorical.columns).index("GENDER")] = "other"

Now if we check the value counts again we can see that the transformation has been applied:

In [32]:
categorical['GENDER'].value_counts(dropna=False)

F        51277
M        39094
other     5041
Name: GENDER, dtype: int64